<a href="https://colab.research.google.com/github/dacarvalho/fsl23/blob/master/fsl23_intro_to_yocto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yocto Basic Setup

## 1. Setting up the build environment

### 1.1. Install Yocto Dependencies


```shell
$ sudo apt install gawk wget git diffstat unzip texinfo gcc build-essential chrpath socat cpio python3 python3-pip python3-pexpect xz-utils debianutils iputils-ping python3-git python3-jinja2 libegl1-mesa libsdl1.2-dev python3-subunit mesa-common-dev zstd liblz4-tool file locales
$ sudo locale-gen en_US.UTF-8
```


### 1.2. Prepare directory

```shell
$ mkdir yocto
$ cd yocto
$ mkdir {builds,layers}
```


### 1.3. Clone poky and other layers

```shell
$ git -C layers/ clone -b kirkstone git://git.yoctoproject.org/poky # Reference distribution and all yocto tools (bitbake, oe)
$ git -C layers/ clone -b kirkstone https://github.com/agherzan/meta-raspberrypi.git # BSP layer for the RaspberryPi
```

### 1.4. Create a new build environment
```shell
$ cd builds
$ source ../layers/poky/oe-init-build-env <nome do diretório da build>
```


## 2. Starting your first build

### 2.1. Add new layers to the build

```shell
$ cat conf/bblayers.conf # Reparar nas layers que são adicionadas por default à build
$ bitbake-layers add-layer ../../layers/meta-raspberrypi
$ cat conf/bblayers.conf # Reparar na nova linha adicionada
```

### 2.2. Choose build target
```shell
$ echo "MACHINE = \"raspberrypi4-64\"" >> conf/local.conf # Para RPi4
$ echo "MACHINE = \"beaglebone-yocto\"" >> conf/local.conf # para Beaglebone
```

### 2.3. Add SSH server for remote access
```shell
$ sed -i 's/\('EXTRA_IMAGE_FEATURES' ?= "\)[^"]*/\1'debug-tweaks\ ssh-server-openssh'/' conf/local.conf
```

### 2.4. Exploring Bitbake
```shell
$ bitbake -c listtasks
$ bitbake -s
$ bitbake -e | less
$ bitbake -v
```

### 2.5. Build test image
```shell
$ bitbake rpi-test-image # Para RaspberryPi
$ bitbake core-image-minimal # Para Beaglebone ou RaspberryPi
```

# Extending Yocto

## 1. Creating a new layer

```shell
$ bitbake-layers create-layer ../../layers/meta-feira
```
``` shell
$ tree
├── conf
│   └── layer.conf
├── COPYING.MIT
├── README
└── recipes-example
    └── example
        └── example_0.1.bb
```
```shell
$ rm -rf recipes-example
$ bitbake-layers add-layer ../../layers/meta-feira
```

## 2. Creating a recipe extension
The `os-release` recipe  is part of the `meta` layer, located on the `recipes-core` directory. So:

```shell
$ mkdir -p recipes-core/os-release
$ touch recipes-core/os-release/os-release.bbappend
$ echo "OS_RELEASE_FIELDS:append = \"MACHINE\"" >> recipes-core/os-release/os-release.bbappend

```

## 3. Developing a Web Server

For the workshop we developed a very simple webserver, presented here. If you want to create your own software, feel free to.

This webserver handles GET requests.

```python
#!/usr/bin/python3

from http.server import BaseHTTPRequestHandler, HTTPServer
import csv
import pathlib

hostName = "0.0.0.0"
serverPort = 8080
path = pathlib.Path("/etc/os-release")
with open(path) as buf:
    reader = csv.reader(buf, delimiter="=")
    os_release = dict(reader)


class MyServer(BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(bytes("<html><head><title>Yocto Workshop</title></head>", "utf-8"))
        self.wfile.write(bytes("<p>Request: %s</p>" % self.path, "utf-8"))
        self.wfile.write(bytes("<body>", "utf-8"))
        self.wfile.write(bytes("<p>This server is running on {machine}</p>".format(machine=os_release.get("MACHINE", "unknown")), "utf-8"))
        self.wfile.write(bytes("</body></html>", "utf-8"))

if __name__ == "__main__":        
    webServer = HTTPServer((hostName, serverPort), MyServer)
    print("Server started http://%s:%s" % (hostName, serverPort))

    try:
        webServer.serve_forever()
    except KeyboardInterrupt:
        pass

    webServer.server_close()
    print("Server stopped.")
```

## 4. Creating a systemd service

We can add a systemd service to launch our webserver on boot.

```systemd
[Unit]
Description=?SERV_DESC?
After=multi-user.target

[Service]
Type=idle
RemainAfterExit=no
ExecStart=/usr/bin/feira-webserver.py

[Install]
WantedBy=multi-user.target

```



## 5. Create a recipe for our software

```bitbake
DESCRIPTION = "Example Web Server for Yocto Workshop"
SUMMARY = "${DESCRIPTION}"
LICENSE = "CLOSED"
LIC_FILES_CHKSUM = ""

inherit systemd

SRC_URI = " \
	file://${BPN}.py \
	file://${BPN}.service \
	"

SYSTEMD_AUTO_ENABLE = "enable"
SYSTEMD_PACKAGES:append = " ${PN}"
SYSTEMD_SERVICE_${PN} = "${PN}.service"

RDEPENDS:${PN} = "python3-core python3-netclient python3-netserver"


do_install () {
    sed -i 's/?\(SERV_DESC\)?/'"${SUMMARY}"'/' ${WORKDIR}/${PN}.service

	install -d ${D}${bindir}/
	install -m 0755 ${WORKDIR}/${PN}.py ${D}${bindir}/

	install -d ${D}/${systemd_unitdir}/system
    install -d ${D}${sysconfdir}/systemd/system/multi-user.target.wants/
    install -m 0644 ${WORKDIR}/${PN}.service ${D}/${systemd_unitdir}/system

	ln -sf ${systemd_unitdir}/system/${PN}.service ${D}${sysconfdir}/systemd/system/multi-user.target.wants/${PN}.service
}

FILES:${PN} += "${systemd_unitdir}/system/*.service ${sysconfdir}"
```

## 6. Update your build

Change the init system to systemd
```bitbake
DISTRO_FEATURES:append = " systemd"
DISTRO_FEATURES_BACKFILL_CONSIDERED = "sysvinit"
VIRTUAL-RUNTIME_init_manager = "systemd"
```
Add your newly created package to `local.conf`
```bitbake
IMAGE_INSTALL:append = " feira-webserver"
```

## 7. Rebuild your images

```shell
$ bitbake rpi-test-image
$ # or
$ bitbake core-image-minimal
```